##### Copyright 2018 Los autores de TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Puntos de control de entrenamiento

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a></td>
</table>

La frase "Guardar un modelo de TensorFlow" generalmente significa una de dos cosas:

1. Puntos de control, O
2. SavedModel.

Los puntos de control capturan el valor exacto de todos los parámetros ( `tf.Variable` ) utilizados por un modelo. Los puntos de control no contienen ninguna descripción del cálculo definido por el modelo y, por lo tanto, generalmente solo son útiles cuando el código fuente que usará los valores de los parámetros guardados está disponible.

El formato SavedModel, por otro lado, incluye una descripción serializada del cálculo definido por el modelo además de los valores de los parámetros (punto de control). Los modelos en este formato son independientes del código fuente que creó el modelo. Por lo tanto, son adecuados para su implementación a través de TensorFlow Serving, TensorFlow Lite, TensorFlow.js o programas en otros lenguajes de programación (las API de C, C ++, Java, Go, Rust, C #, etc. TensorFlow).

Esta guía cubre las API para escribir y leer puntos de control.

## Preparar

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## Guardar desde las API de entrenamiento de `tf.keras`

See the [`tf.keras` guide on saving and restoring](./keras/overview.ipynb#save_and_restore).

`tf.keras.Model.save_weights` guarda un punto de control de TensorFlow. 

In [0]:
net.save_weights('easy_checkpoint')

## Escribir puntos de control


The persistent state of a TensorFlow model is stored in `tf.Variable` objects. These can be constructed directly, but are often created through high-level APIs like `tf.keras.layers` or `tf.keras.Model`.

La forma más sencilla de administrar variables es adjuntarlas a objetos de Python y luego hacer referencia a esos objetos.

Las subclases de `tf.train.Checkpoint` , `tf.keras.layers.Layer` y `tf.keras.Model` un seguimiento automático de las variables asignadas a sus atributos. El siguiente ejemplo construye un modelo lineal simple, luego escribe puntos de control que contienen valores para todas las variables del modelo.

You can easily save a model-checkpoint with `Model.save_weights`

### Punto de control manual

#### Preparar

Para ayudar a demostrar todas las características de `tf.train.Checkpoint` defina un conjunto de datos de juguete y un paso de optimización:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### Crea los objetos del punto de control

To manually make a checkpoint you will need a `tf.train.Checkpoint` object. Where the objects you want to checkpoint are set as attributes on the object.

Un `tf.train.CheckpointManager` también puede ser útil para administrar múltiples puntos de control.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### Entrene y controle el modelo

El siguiente ciclo de entrenamiento crea una instancia del modelo y de un optimizador, luego los reúne en un objeto `tf.train.Checkpoint` . Llama al paso de entrenamiento en un bucle en cada lote de datos y escribe puntos de control periódicamente en el disco.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### Restaurar y continuar entrenando

Después del primero, puede aprobar un nuevo modelo y gerente, pero retome el entrenamiento exactamente donde lo dejó:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

El objeto `tf.train.CheckpointManager` elimina los puntos de control antiguos. Arriba, está configurado para mantener solo los tres puntos de control más recientes.

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

Estas rutas, por ejemplo, `'./tf_ckpts/ckpt-10'` , no son archivos en el disco. En su lugar, son prefijos para un archivo de `index` y uno o más archivos de datos que contienen los valores de las variables. Estos prefijos se agrupan en un solo archivo de `checkpoint` ( `'./tf_ckpts/checkpoint'` ) donde el `CheckpointManager` guarda su estado.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## Mecánica de carga

TensorFlow matches variables to checkpointed values by traversing a directed graph with named edges, starting from the object being loaded. Edge names typically come from attribute names in objects, for example the `"l1"` in `self.l1 = tf.keras.layers.Dense(5)`. `tf.train.Checkpoint` uses its keyword argument names, as in the `"step"` in `tf.train.Checkpoint(step=...)`.

El gráfico de dependencia del ejemplo anterior se ve así:

![Visualización del gráfico de dependencia para el ciclo de entrenamiento de ejemplo](https://tensorflow.org/images/guide/whole_checkpoint.svg)

Con el optimizador en rojo, las variables regulares en azul y las variables de ranura del optimizador en naranja. Los otros nodos, por ejemplo, los que representan `tf.train.Checkpoint` , son negros.

Las variables de ranura son parte del estado del optimizador, pero se crean para una variable específica. Por ejemplo, los bordes `'m'` arriba corresponden al impulso, que el optimizador de Adam rastrea para cada variable. Las variables de ranura solo se guardan en un punto de control si la variable y el optimizador se guardarían ambos, por lo tanto, los bordes discontinuos.

Llamar a `restore()` en un objeto `tf.train.Checkpoint` cola las restauraciones solicitadas, restaurando los valores de las variables tan pronto como hay una ruta coincidente desde el objeto `Checkpoint` . Por ejemplo, podemos cargar solo el sesgo del modelo que definimos anteriormente reconstruyendo una ruta hacia él a través de la red y la capa.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

El gráfico de dependencia para estos nuevos objetos es un subgráfico mucho más pequeño del punto de control más grande que escribimos anteriormente. Incluye solo el sesgo y un contador de guardado que `tf.train.Checkpoint` usa para numerar los puntos de control.

![Visualización de un subgrafo para la variable de sesgo](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` devuelve un objeto de estado, que tiene aserciones opcionales. Todos los objetos que hemos creado en nuestro nuevo `Checkpoint` han sido restaurados, por lo que `status.assert_existing_objects_matched()` pasa.

In [0]:
status.assert_existing_objects_matched()

There are many objects in the checkpoint which haven't matched, including the layer's kernel and the optimizer's variables. `status.assert_consumed()` only passes if the checkpoint and the program match exactly, and would throw an exception here.

### Restauraciones retrasadas

`Layer` objects in TensorFlow may delay the creation of variables to their first call, when input shapes are available. For example the shape of a `Dense` layer's kernel depends on both the layer's input and output shapes, and so the output shape required as a constructor argument is not enough information to create the variable on its own. Since calling a `Layer` also reads the variable's value, a restore must happen between the variable's creation and its first use.

Para admitir este idioma, `tf.train.Checkpoint` restaura las colas que aún no tienen una variable coincidente.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### Inspección manual de puntos de control

`tf.train.list_variables` enumera las claves del punto de control y las formas de las variables en un punto de control. Las claves de los puntos de control son rutas en el gráfico que se muestra arriba.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Seguimiento de listas y diccionarios

Al igual que con las asignaciones directas de atributos como `self.l1 = tf.keras.layers.Dense(5)` , la asignación de listas y diccionarios a los atributos hará un seguimiento de su contenido.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Es posible que observe objetos contenedores para listas y diccionarios. Estos contenedores son versiones con puntos de control de las estructuras de datos subyacentes. Al igual que la carga basada en atributos, estos contenedores restauran el valor de una variable tan pronto como se agrega al contenedor.

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

The same tracking is automatically applied to subclasses of `tf.keras.Model`, and may be used for example to track lists of layers.

## Guardar puntos de control basados en objetos con Estimator

See the [Estimator guide](https://www.tensorflow.org/guide/estimator).

Estimators by default save checkpoints with variable names rather than the object graph described in the previous sections. `tf.train.Checkpoint` will accept name-based checkpoints, but variable names may change when moving parts of a model outside of the Estimator's `model_fn`. Saving object-based checkpoints makes it easier to train a model inside an Estimator and then use it outside of one.

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` puede cargar los puntos de control del Estimador desde su `model_dir` .

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Resumen

Los objetos de TensorFlow proporcionan un mecanismo automático sencillo para guardar y restaurar los valores de las variables que utilizan.
